In [2]:
!pip install transformers
!pip install datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoModel
from transformers import DataCollatorWithPadding
from datasets import load_dataset
from torch.utils.data import DataLoader
import pandas as pd
from transformers import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


2022-11-22 18:49:28.406747: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 18:49:29.562332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-22 18:49:29.562378: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-22 18:49:29.562383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
raw_train_dataset = load_dataset('csv', data_files='cople_ortho.csv', split='train[:80%]')
raw_validation_dataset = load_dataset('csv', data_files='cople_ortho.csv', split='train[80%:]')
labels = pd.read_csv('cople_ortho.csv')['Proficiency'].unique().tolist()
print(labels)

Using custom data configuration default-697819366a407adc
Found cached dataset csv (/home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-697819366a407adc
Found cached dataset csv (/home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


['A2', 'B1', 'B2', 'A1', 'C1']


In [4]:
checkpoint = 'neuralmind/bert-base-portuguese-cased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [5]:
print(raw_train_dataset)
print(raw_validation_dataset)

Dataset({
    features: ['Id', 'Proficiency', 'Text'],
    num_rows: 816
})
Dataset({
    features: ['Id', 'Proficiency', 'Text'],
    num_rows: 204
})


In [6]:
def class_string_to_int(s):
    if s == 'A1':
        return 0
    if s == 'A2':
        return 1
    if s == 'B1':
        return 2
    if s == 'B2':
        return 3
    if s == 'C1':
        return 4
    if s == 'C2':
        return 5

In [7]:

def tokenize_function(example):
    example['labels'] = [class_string_to_int(proficiency) for proficiency in example['Proficiency']]
    return tokenizer(example["Text"], truncation=True, max_length=512)


tokenized_train_datasets = raw_train_dataset.map(tokenize_function, batched=True)
tokenized_validation_datasets = raw_validation_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-647d7d12b977d667.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
print(tokenized_train_datasets)
print(tokenized_validation_datasets)

Dataset({
    features: ['Id', 'Proficiency', 'Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 816
})
Dataset({
    features: ['Id', 'Proficiency', 'Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 204
})


In [9]:
tokenized_train_datasets = tokenized_train_datasets.remove_columns(["Text", "Id", "Proficiency"])
tokenized_train_datasets.set_format("torch")
tokenized_train_datasets.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [10]:
tokenized_validation_datasets = tokenized_validation_datasets.remove_columns(["Text", "Id", "Proficiency"])
tokenized_validation_datasets.set_format("torch")
tokenized_validation_datasets.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [18]:
train_dataloader = DataLoader(
    tokenized_train_datasets, shuffle=True, batch_size=4, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_validation_datasets, batch_size=4, collate_fn=data_collator
)

In [19]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([4]),
 'input_ids': torch.Size([4, 279]),
 'token_type_ids': torch.Size([4, 279]),
 'attention_mask': torch.Size([4, 279])}

In [20]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [14]:
optimizer = AdamW(model.parameters(), lr=5e-5)

/home/joao/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

306


In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


device(type='cuda')

In [17]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/306 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.82 GiB total capacity; 2.73 GiB already allocated; 14.06 MiB free; 2.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import evaluate

# metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()